<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_LiveloEsfera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# 1. Instala√ß√£o de pacotes necess√°rios
!pip install selenium
!apt-get update # para o ChromeDriver
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 384 kB in 1s (274 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [39]:
# 2. Importa√ß√µes
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

In [40]:
# 3. Configura√ß√µes do Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

In [41]:
# 4. Fun√ß√£o para extrair dados de uma p√°gina
def extrair_dados_amazon_livelo():
    driver.get("https://www.livelo.com.br/juntar-pontos/parceiros/amazon/")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    texto = soup.get_text()
    return "Resumo Livelo:\n" + texto[:1000]  # limita para exemplo

In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def extrair_esfera():
    options = Options()
    options.add_argument("--headless")  # Remova esta linha para debugar com navegador vis√≠vel
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)

    try:
        url = "https://www.esfera.com.vc/p/amazon/e000100298"
        driver.get(url)

        # Espera o carregamento do t√≠tulo como indicativo de que o conte√∫do principal foi carregado
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-title")))

        # Agora usa BeautifulSoup para extrair os campos HTML indicados
        soup = BeautifulSoup(driver.page_source, "html.parser")

        titulo = soup.find("h1", class_="product-title").text.strip()

        beneficio_section = soup.find("p", string="Beneficio")
        beneficio_div = beneficio_section.find_next("div")
        beneficio_raw = beneficio_div.contents[0].strip()
        beneficio_info = beneficio_div.find("p").text.strip()

        validade_section = soup.find("p", string="Validade")
        validade_text = validade_section.find_next("div").text.strip()

        credito_section = soup.find("p", string="Creditado em")
        credito_text = credito_section.find_next("div").text.strip()

        resultado = f"""
üì¶ Benef√≠cio {titulo} = {beneficio_raw} ({beneficio_info})
üóìÔ∏è Validade da promo√ß√£o: At√© {validade_text}
üí≥ Pontos creditados em at√© {credito_text}
"""
    except Exception as e:
        resultado = f"‚ùå Erro ao extrair dados da Esfera: {str(e)}"
    finally:
        driver.quit()

    return resultado.strip()

In [44]:
esfera = extrair_esfera()
esfera

'üì¶ Benef√≠cio Amazon = 1x1 (Ganhe 1 ponto a cada real)\nüóìÔ∏è Validade da promo√ß√£o: At√© 31/12/25\nüí≥ Pontos creditados em at√© 90 dias do recebimento do produto'

Enviar E-mail

In [45]:
#importa√ß√µes
from email.message import EmailMessage
import os
import smtplib

#definindo dados de envio do e-mail
remetente = os.environ['EMAIL_REMETENTE']
destinatario = os.environ['EMAIL_DESTINATARIO']
assunto = 'Seu resumo di√°rio pontos Amazon'
mensagem = f"""Ol√°, aqui est√° o seu resumo dos pontos Amazon:

{esfera}"""

senha = os.environ['SENHA_GMAIL']

#crinado o e-mail
msg = EmailMessage()
msg['Subject'] = assunto
msg['From'] = remetente
msg['To'] = destinatario
msg.set_content(mensagem)

#enviando o e-mail
with smtplib.SMTP_SSL("smtp.gmail.com",465) as email:
  email.login(remetente,senha)
  email.send_message(msg)

print("O email foi enviado com sucesso!")

KeyError: 'EMAIL_REMETENTE'